In [1]:
#学号：2020217793 姓名：洪祎麟 班级：电信科20-1班

In [ ]:
#本次作业的第二题我选择了电商网站京东中“R9000P电脑”的前30页评论进行提取。

#通过运用一系列爬虫知识，爬下来前30页所有关于此电脑的评论。具体的爬虫内容包括评论者的id,评论者的具体评论内容和评论者的评论时间

#最后，将提取出来的所有数据存储入列表中，输出为xlsx格式。

In [3]:
import requests
import json
import pandas as pd #为头文件

<Response [200]>


In [ ]:
def get_html(url,header):  #定义函数 该函数用于获取网页里面的html信息
    try:          #try except异常处理机制，若失败则直接返回“网页访问出错”，防止代码直接崩溃
        r=requests.get(url,headers=header)  #用requests的get方法获取网址
        r.raise_for_status() #用于判断访问网络的状态（正常即为200）
        r.encoding=r.apparent_encoding #改变提取出的编码为正确的编码形式.0
        return r.text #将从网页提取出的东西返回为文本文件
    except: #try except异常处理机制同上。此处为异常出现的情况
        print("网页访问出错") #则输出访问出错，返回None
        return None

In [ ]:
def get_pdata(data):  #定义函数 该函数通过使用Json库，筛选出所需要的值（购买者id，购买者的评论，购买时间）并返回值列表中
    lst=[] #定义一个空列表
    jd = json.loads(data.lstrip('fetchJSON_comment98vv12345(').rstrip(');')) #通过这一筛选，把之前网页中提取的东西筛选成有用值，筛掉所有无意义的字符串
    data_list = jd['comments'] #由于代码的结构中'comments'里面包含了我们所需要的三个值，故直接将它用json的方法提取出来，以以致于后面的提取
    for data in data_list: #循环，提取出一段中所有的购买者id，购买时间和购买者的评论
        id = data['id'] #提取id
        conment = data['content']  #提取评论
        time = data['creationTime'] #提取评论时间
        lst.append([id, conment, time])  # 将以上三个提取出的元素一同放入之前建的lst这一列表中
    return lst #返回该列表便于之后使用

In [ ]:
def climbdata(): #定义函数，该函数用于爬取网页的所有页数来爬取数据
    datalst=[]  #定义一个空列表，之后用于存储数据
    for n in range(1,31): #遍历循环，遍历所有网页的前30页，便于爬取数据
        url=f'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=100010793473&score=0&sortType=5&page={n}&pageSize=10&isShadowSku=0&fold=1' #需要爬取数据的网页，用{n}即是为了通过循环来爬取每一页
        print(f"现在已经下载到：第{n}页了")  #输出正在下载页数的标识
        html=get_html(url,header) #通过之前的函数，获得具体某一网页的全部文本
        if html is None: #如果网页没有东西，直接访问下一页
            continue
        lst=get_pdata(html) #通过运用这一函数，筛选出所需的三项数据。
        datalst.extend(lst) #把之前筛选出的数据存入datalst之后（注意需要使用extend方法接在后面）
    print("下载结束啦！")   #全部下载完毕后输出标识
    return datalst #返回存储数据的列表

In [ ]:
header={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}
#为网站的头，虚拟自己的浏览器环境，便于让爬虫顺利爬取数据

In [ ]:
list=climbdata() #将爬虫爬下来的数据存入list中
df=pd.DataFrame(list,columns=['购买者id','评论','发布时间']) #将列表转换为DataFrame格式
df.to_excel('1002.xlsx')  #将DataFrame格式最后转化为xlsx格式输出